In [1]:
from atl_module import kriging
import geopandas as gpd
from atl_module import GebcoUpscaler
import rasterio as rio
from shapely.geometry import box

In [2]:
%cd ..


/mnt/c/Users/maxli/OneDrive - Van Oord/Documents/thesis/code


In [3]:
jarkuspts = gpd.read_file('../data/jarkus/jarkus-2021_Noord-Holland.gpkg')


In [4]:
jarkuspts = jarkuspts.assign(X=jarkuspts.geometry.x,Y=jarkuspts.geometry.y)

In [5]:
truth = rio.open('../data/special_sites/petten/in-situ-DEM/truth.vrt')
truthrascrs= truth.crs
truth_ras_bounds = truth.bounds

In [6]:
subset_jarkus = jarkuspts.to_crs(truthrascrs).clip(truth_ras_bounds).to_crs(jarkuspts.estimate_utm_crs())
# add the location in local UTM system as a columns
subset_jarkus = subset_jarkus.assign(northing=subset_jarkus.geometry.y,easting=subset_jarkus.geometry.x,sf_elev_MSL=subset_jarkus.geometry.z).loc[:,['X','Y','northing','easting','sf_elev_MSL','geometry']]
sub_jarkus = subset_jarkus[subset_jarkus.sf_elev_MSL < 1]

In [7]:
subset_jarkus.to_file('../data/special_sites/petten/all_bathy_pts.gpkg')

In [8]:
site = GebcoUpscaler('../data/special_sites/petten','petten','../data/special_sites/petten/in-situ-DEM/truth.vrt')

In [9]:
site.subset_gebco(50)

[D 220918 16:41:09 raster_interaction:128] GEBCO subset raster written to ../data/special_sites/petten/bilinear.tif, with CRS EPSG:32631


In [ ]:
%%time
site.kriging(
    kr_model='uk',
    npts=2000,
    variogram_parameters={'range':10000,'nugget':0.7,'sill':23},
    samplemethod='dart'
    )

[I 220918 16:41:09 core:194] Kriging petten site using 2000 points with crs epsg:32631 with options {'variogram_parameters': {'range': 10000, 'nugget': 0.7, 'sill': 23}, 'samplemethod': 'dart'}
[D 220918 16:41:10 kriging:23] The types of the numpy array that is sent to PDAL poisson dart throwing are: (numpy.record, [('Y', '<f8'), ('X', '<f8'), ('Z', '<f8')])
[D 220918 16:41:10 kriging:29] 2000 points remaining after relaxation dart throwing culling
[D 220918 16:41:12 kriging:71] 2000 points selected with dart written to ../data/special_sites/petten/kriging_pts


Adjusting data for anisotropy...
Initializing variogram model...
Using 'spherical' Variogram Model
Partial Sill: 22.3
Full Sill: 23.0
Range: 10000
Nugget: 0.7 

Calculating statistics on variogram model fit...


In [ ]:
site.kalman_update(0.75,)
site.raster_rmse(check_kriged=True)

In [ ]:
site.kalman_update(1)
site.raster_rmse()

In [ ]:
site.kalman_update(1.5)
site.raster_rmse()

In [ ]:
site.kalman_update(2)
site.raster_rmse()

In [ ]:
site.kalman_update(3)
site.raster_rmse()

In [ ]:
site.bathy_pts_gdf = site.bathy_pts_gdf.assign(easting_corr=0,northing_corr=0)
site.add_truth_data()

In [ ]:
site.lidar_error()

In [ ]:
site.plot_lidar_error()

In [ ]:
site.write_error_tables()